In [1]:
!pip install fake_useragent

In [2]:
import re
import csv
import time
import json
import random
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm
import concurrent.futures
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from concurrent.futures import ThreadPoolExecutor, as_completed

In [3]:
# Подключение к google drive

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


### Рейтинг банков - https://www.banki.ru/services/responses/

Результаты будем сохранять в файл **banki_banks_{year}.csv**.

* place - место в рейтинге
* name - название банка (на русском языке)
* rating - рейтинг
* responses - кол-во отзывов
* answers - кол-во ответов банка

In [4]:
%%time

for year in [2023, 2024]:

  df = pd.DataFrame({'place':[], 'name':[], 'rating':[], 'responses':[], 'answers':[]})

  page = 1
  i = 1
  while True:

    url = f'https://www.banki.ru/services/responses/?date={year}&page={page}'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    items = soup.find_all("script", {"type": "application/ld+json"})

    if len(items) == 1:
      break

    for item in items[:50]:
      data = json.loads(item.text)
      if 'name' in data:
        place = i
        name = data['name']
        rating = data['aggregateRating']['ratingValue']
        responses = data['aggregateRating']['ratingCount']
        answers = data['aggregateRating']['reviewCount']

        df.loc[len(df)] = [i, name, rating, responses, answers]
        i += 1
    page += 1

  df.to_csv(f'/content/drive/MyDrive/bank_reviews_nlp/banki_banks_{year}.csv', index=False)

CPU times: user 4.01 s, sys: 89.5 ms, total: 4.1 s
Wall time: 19 s


### Отзывы о банках - https://www.banki.ru/services/responses/list/?type=all

Парсить отзывы на банки будем в 2 этапа: сначала ссылки на отзывы, затем будем переходить по каждой их них и парсить сам отзыв.

Для выбора страниц необходимо вручную найти нужный временной диапазон. Для примера спарсим ссылки с первых 20 страниц.

In [5]:
# Ссылки на отзывы - однопоточный

%%time

base_url = 'https://www.banki.ru'

with open('/content/drive/MyDrive/bank_reviews_nlp/banki_ru_urls.csv', "w", encoding='utf-8') as w_file:
  file_writer = csv.writer(w_file, delimiter = ",", lineterminator="\n")
  file_writer.writerow(['url'])

  for page in tqdm(range(1,21)):

    url = f'https://www.banki.ru/services/responses/list/?type=all&page={page}'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')


    items = soup.find_all('a', attrs={'class': 'link-simple',
                                      'data-gtm-click': '{"event":"GTM_event","eventCategory":"ux_data","eventAction":"click_responses_response_user_rating_banking_allReviewsPage"}'})

    for item in items:

      url_resp = base_url+item['href']

      file_writer.writerow([url_resp])

    if page % 10 == 0:
      w_file.flush()

100%|██████████| 20/20 [00:42<00:00,  2.14s/it]

CPU times: user 6.67 s, sys: 114 ms, total: 6.78 s
Wall time: 42.9 s


Для сокращение времени будем использовать многопоточность.



Хорошая статья про библиотеку concurrent.futures, которую мы будем использовать
https://habr.com/ru/companies/otus/articles/771346/.

In [6]:
# Ссылки на отзывы - многопоточный

%%time

ua = UserAgent()
user_agents = [ua.chrome, ua.google, ua['google chrome'], ua.firefox, ua.ff, ua.safari]

def get_urls(url):

  headers = {'User-Agent': random.choice(user_agents)}
  response = requests.get(url, headers=headers, timeout=30)
  soup = BeautifulSoup(response.content, 'html.parser')

  items = soup.find_all('a', attrs={'class': 'link-simple',
                                    'data-gtm-click': '{"event":"GTM_event","eventCategory":"ux_data","eventAction":"click_responses_response_user_rating_banking_allReviewsPage"}'})

  urls = []
  for item in items:
    urls.append('https://www.banki.ru' + item['href'])

  return urls


def write_to_csv(urls):
  with open('/content/drive/MyDrive/bank_reviews_nlp/banki_ru_urls.csv', "a", encoding='utf-8') as w_file:
    file_writer = csv.writer(w_file, delimiter=",", lineterminator="\n")
    for url in urls:
      file_writer.writerow([url])


urls = [f'https://www.banki.ru/services/responses/list/?type=all&page={page}' for page in range(1, 21)]

with open('/content/drive/MyDrive/bank_reviews_nlp/banki_ru_urls.csv', "a", encoding='utf-8') as w_file:
    file_writer = csv.writer(w_file, delimiter=",", lineterminator="\n")
    file_writer.writerow(['url'])

err = open('/content/drive/MyDrive/bank_reviews_nlp/errors.txt', 'a')

with ThreadPoolExecutor(max_workers=8) as executor:
  futures = [executor.submit(get_urls, url) for url in urls]
  for future in tqdm(as_completed(futures), total=len(urls)):
    try:
      results = future.result()
      add_row = True
    except Exception as exc:
      add_row = False
      err.write(str(type(exc)) + '\n')
    finally:
      if add_row == True:
        write_to_csv(results)
      else:
        pass

w_file.close()
err.close()

100%|██████████| 20/20 [00:10<00:00,  1.95it/s]

CPU times: user 8 s, sys: 198 ms, total: 8.2 s
Wall time: 10.3 s


Видно, что в результате использования многопоточности получилось уменьшить время в ~4 раза.

Для скачивания ссылок за 2023 год (полностью) и 1 квартал 2024 года понадобилось около 3.5 часов против 15 часов, без использования многопоточности.
Количество отзывов - 477 тысяч.

Далее будем переходить по этим ссылкам и парсить отзывы.

Результаты будем сохранять в файл - **banki_ru_reviews.csv**.

* url - ссылка на отзыв
* date_review - дата оставления отзыва
* time_review - время оставления отзыва
* user_name - имя пользователя
* user_city - город пользователя
* review_title - тема обращения
* review_text - текст обращения
* review_status - статус обращения (проверяется/проверен)
* rating - оценка
* clear_conditions_rating - оценка прозрачности условий
* polite_staff_rating - оценка вежливости сотрудников
* support_rating - оценка доступности и поддержки
* app_site_rating - оценка удобства приложения, сайта
* bank_name - назание банка (на русском языке)
* is_bank_ans - наличие ответа банка (yes/no)
* time_bank_ans - время оставления ответа банка
* date_bank_ans - дата оставления ответа банка
* bank_text_ans - текст ответа банка

In [7]:
%%time

ua = UserAgent()
user_agents = [ua.chrome, ua.google, ua['google chrome'], ua.firefox, ua.ff, ua.safari]

def get_review(url):

  headers = {'user-Agent': random.choice(user_agents)}
  response = requests.get(url, headers=headers, timeout=30)
  soup = BeautifulSoup(response.content, 'html.parser')

  date_review = soup.find('span', {'class':'l10fac986'}).text[4:14]
  time_review = soup.find('span', {'class':'l10fac986'}).text[15:20]
  user_name = soup.find('span', {'class':'l17191939'}).text.strip()
  user_city = soup.find('span', {'class':'l3a372298'}).text
  review_title = soup.find('h1', {'class':'text-header-0 le856f50c'}).text.strip()
  review_text = soup.find("div", {"class":"lf4cbd87d ld6d46e58 lfd76152f"}).text.strip()
  rating = soup.find('div', {'class':'lbb810226'}).text.strip()
  if rating == 'Без оценки':
    review_status = 'unk'
  else:
    review_status = soup.find('section', {'class':'lf4cbd87d l9656ec89 lfd76152f'}).text.strip()

  additional_grades = {}
  for txts6 in soup.find_all('div', {"class": 'text-size-6'}):
      additional_grades[txts6.text] = 0
  i = 0
  for grade in soup.find_all('div', {'class': 'ld017b199'}):
      current_key = list(additional_grades.keys())[i]
      additional_grades[current_key] = str(grade).count("l61f54b7b")
      i += 1
  clear_conditions_rating = additional_grades.get('Прозрачные условия', 0)
  polite_staff_rating = additional_grades.get('Вежливые сотрудники', 0)
  support_rating = additional_grades.get('Доступность и поддержка', 0)
  app_site_rating = additional_grades.get('Удобство приложения, сайта', 0)

  bank_name = soup.find('img', {'class':'lazy-load'})['alt']
  bank_ans = soup.find('div', {'class':'l0e7bcaa7'})
  if bank_ans is None:
    is_bank_ans = 'no'
    date_bank_ans = 'unk'
    time_bank_ans = 'unk'
    bank_text_ans = 'unk'
  else:
    is_bank_ans = 'yes'
    date_bank_ans = soup.find('div', {'class':'l0e7bcaa7'}).find('div', {'class':'l46c44745'}).text[:10].strip()
    time_bank_ans = soup.find('div', {'class':'l0e7bcaa7'}).find('div', {'class':'l46c44745'}).text[11:19].strip()
    bank_text_ans = soup.find('div', {'class':'l0e7bcaa7'}).find('div', {'class':'lb1789875'}).text.strip()

  review_row = [url, date_review, time_review, user_name, user_city, review_title, review_text, review_status,
               rating, clear_conditions_rating, polite_staff_rating, support_rating, app_site_rating,
               bank_name, is_bank_ans, time_bank_ans, date_bank_ans, bank_text_ans]

  return review_row


def write_to_csv(review_row):
  with open('/content/drive/MyDrive/bank_reviews_nlp/banki_ru_reviews.csv', "a", encoding='utf-8') as w_file:
    file_writer = csv.writer(w_file, delimiter=",", lineterminator="\n")
    file_writer.writerow(review_row)

urls = pd.read_csv('/content/drive/MyDrive/bank_reviews_nlp/banki_ru_urls.csv')
urls = urls.iloc[:,0].tolist()

with open('/content/drive/MyDrive/bank_reviews_nlp/banki_ru_reviews.csv', "w", encoding='utf-8') as w_file:
  file_writer = csv.writer(w_file, delimiter = ",", lineterminator="\n")
  file_writer.writerow(['url', 'date_review', 'time_review', 'user_name', 'user_city', 'review_title', 'review_text', 'review_status',
                        'rating', 'clear_conditions_rating', 'polite_staff_rating', 'support_rating', 'app_site_rating',
                        'bank_name', 'is_bank_ans', 'time_bank_ans', 'date_bank_ans', 'bank_text_ans'])

err = open('/content/drive/MyDrive/bank_reviews_nlp/errors_2.txt', 'a')

with ThreadPoolExecutor(max_workers=8) as executor:
  futures = [executor.submit(get_review, url) for url in urls]
  for future in tqdm(as_completed(futures), total=len(urls)):
    try:
      results = future.result()
      add_row = True
    except Exception as exc:
      add_row = False
      err.write(str(type(exc)) + '\n')
    finally:
      if add_row == True:
        write_to_csv(results)
      else:
        pass

w_file.close()
err.close()

100%|██████████| 1001/1001 [05:26<00:00,  3.06it/s]

CPU times: user 5min 4s, sys: 5.25 s, total: 5min 9s
Wall time: 5min 27s


Для скачивания всех отзывов понадобилось около 45 часов. Так как у colab есть ограничение в 12 часов, отзывы парсились в несколько разных файлов, а затем объединялись.